## Using SQLModel to work with database tables

# <span style="color:red">clear all output before saving: db output contains passwords! </span>

#### Overview

this walks through process of

 - importing modules needed
 - creates a temporary db using .env

#### Requirements

- Postgresql database available and has an example database in it (local or remote)
- the URL for that database is in `../.env` or in the environment
- example data file [../data/test_stations.tsv](../data/test_stations.tsv)


In [ ]:

from ewxpwsdb.db.models import WeatherStation, Reading, StationType, APIResponse
from ewxpwsdb.db.importdata import import_station_file, read_station_table

from tempfile import NamedTemporaryFile


### Create temp Postgresql db

this notebook does not use the .env file or the get_db_url() function for the app's db, 
instead it creates a new file to be used for a temporary, empty postgresql database

Requires that postgresql is running on local host and does not require a password.   

see src/ewxpwsdb/db/database.py tmp_pg_engine function for details, or modify this function to use an external DB

In [ ]:
from ewxpwsdb.db.database import Session, init_db, create_temp_pg_engine, get_db_url, drop_pg_db

# create_temp_pg_engine(admin_db_url:str, name_prefix:str='' )->Engine:

# this creates a db inside the server running on localhost
db_url = get_db_url()
engine = create_temp_pg_engine(admin_db_url=db_url, name_prefix='ewxpws_testdb')
temp_db_name = engine.url.database
temp_db_host = engine.url.host


In [ ]:

init_db(engine)

In [ ]:
# does the database have the tables we expect?

our_table_names = ['WeatherStation', 'Reading', 'apiresponse', 'stationtype']
from sqlalchemy import inspect
inspector = inspect(engine)

db_tables = list(inspector.get_table_names())
db_tables

In [ ]:
# Check the columns in the database
from sqlalchemy import inspect
reading_inspector = inspect(Reading)
print(dir(reading_inspector))
print(reading_inspector.columns)
# insp.columns.name
print(reading_inspector.columns[0])

In [ ]:
from ewxpwsdb.db.models import WeatherStation
station_inspector = inspect(WeatherStation)
colnames = set()
for c in station_inspector.columns:
    colnames.add(c.name)

print(colnames)

### Working with the Station table

In [ ]:
WeatherStation.model_fields

In [ ]:
# import data
# running this twice should not affect the station table or raise an error (only warnings)
station_file = '../data/test_stations.tsv'
import_station_file(station_file, engine = engine)


In [ ]:
from sqlmodel import select
with Session(engine) as session:
        statement = select(WeatherStation)
        results = session.exec(statement)
        stations = results.all()

len(stations)

In [ ]:
# what is the first station? 
station = stations[0]
print(station)

In [ ]:
# attempt to add a duplicate
from sqlalchemy import exc
# from sqlalchemy import IntegrityError
station_list = read_station_table(station_file)
dup_ws = WeatherStation.model_validate(station_list[2])
with Session(engine) as session:
    try:
        session.add(dup_ws)
        session.commit()
        print("if this is printing, then dup records were able to be inserted into the db")
    except exc.IntegrityError as e:
        print("duplicate record caused integrity error, which is what we want!")

    


### Example SQL

In [ ]:
from sqlmodel import text

sql="select * from weatherstation inner join stationtype on weatherstation.station_type = stationtype.station_type where stationtype.station_type = 'ZENTRA';"
with Session(engine) as session:
    stmt = text(sql)
    results = session.exec(stmt)
    records = results.all()

len(records)


### Examing DB properties

### Work with Station Model

### Clean up

remove test database

In [ ]:
session.close()
engine.dispose()
from ewxpwsdb.db.database import drop_pg_db
db_deleted = drop_pg_db(db_name_to_delete=temp_db_name, admin_db_url=db_url)
